(stereographic-subsetting)=
# Using Subset on Data With Original Grids

The **geographical grid**, based on longitude and latitude, is widely used for mapping and navigation. However, as one approaches **±90 degrees latitude**, this projection methods introduces significant distortions at higher latitudes and singularities at the poles. To address this, a commonly used alternative is the stereographic projection, which preserves local shapes and angles while avoiding the ±90-degree singularities. This projection is used in computational models that produce Arctic data and is therefore referred to as the original format, grid, or coordinate system for Arctic data. The specific projection used for each dataset is detailed in the product documentation, which can be explored in the [Copernicus Marine Data Store](https://data.marine.copernicus.eu/products).

In many cases, data is reprojected onto an **equispaced latitude-longitude grid** for consistency. However, when applied to stereographic data, this reprojection can introduce distortions and reduce the relative weight of data at lower latitudes. To mitigate this issue, the data is provided in both its original stereographic format and a reprojected version.

This documentation demonstrates how to use the Copernicus Marine Toolbox to access and work with datasets in both the original and reprojected formats.

## Zones Delimitations in Different Grids

For the datasets are provided in a longitude/latitude projection, areas can be selected with subset using the following options: 
- **Longitude range**: minimum_longitude, maximum_longitude 
- **Latitude range**: minimum_latitude, maximum_latitude 

For the datasets are available in their **original grid**, like the stereographic one, areas can be selected with subset using the grid-specific coordinates along the x and y axes: 

In python:  

  - **X-axis rang**e: minimum_x , maximum_x  
  - **Y-axis range**: minimum_y , maximum_y  

In the CLI: 

  - **X-axis range**: --minimum-x, --maximum-x 
  - **Y-axis range**: --minimum-y, --maximum-y 

In the CLI, those commands also have short forms: 

  - **X-axis range**: -x, -X 
  - **Y-axis range**: -y, -Y 


The s**hort form can be used for both the longitude and latitude projection and the original Grid**. The units will therefore be in the original coordinate system unit if the dataset part chosen is the original one or in latitude for the y range and longitude for the x range if the dataset chosen is the one in longitude and latitude projection. The **default dataset part** is the one in **longitude and latitude projection.** 

These options allow users to subset data directly in the native grid of the dataset, preserving accuracy and avoiding the distortions that can occur when transforming data into a longitude/latitude format. Understanding the coordinate system of a dataset is essential to selecting the correct bounds for subsetting and ensuring precise data extraction.

It should be noted that dataset in stereographic coordinates still have longitudes and latitudes, but they are then **variables and not coordinates.**  


## Accessing Different Grids via Subset

For a given dataset and version, data provided in stereographic projection is stored in a **separate dataset part.** 

As a result, when subsetting data using the **original grid**, it is essential to explicitly specify the correct dataset part. This ensures that the subset operation is performed on the appropriate projection. 

  - In the Python interface, the corresponding parameter is specified as: 

      dataset_part="originalGrid" 

  - In the command-line interface (CLI), this is done using the option:

      --dataset-part originalGrid
    
Clearly defining the dataset part helps avoid confusion and ensures that data is retrieved in the intended projection. 
  

## Units in the Original Coordinate System

When working with original-grid datasets, it is essential to be aware of **significant variations in unit conventions**. These differences can impact data interpretation and subsequent analysis.

### Unit Discrepancies 

The spatial bounds of datasets may be represented using different units, which can affect coordinate system understanding and data processing:

   - Some datasets define spatial extents in **meters**, with coordinate ranges such as -5,500,000 to 5,500,000.

   - Other datasets utilize a unit scale of **100 kilometers**, with bounds appearing in a format such as -28 to 43.

Understanding these variations is critical to ensuring consistency in data handling and visualization.

### Methods for Verifying Unit Conventions

To accurately determine the unit system used within a dataset, the following verification methods are recommended:

   - Inspect the coordinates_extent field in subset responses to confirm the spatial range and unit system.

   - Utilize the describe command to review dataset metadata prior to performing subsetting operations, ensuring compatibility with analytical workflows.

### Using the Toolbox to Verify Unit Conventions

To verify that the units are the ones expected when working with original-grid datasets, it is recommended to inspect the response output using the subset and describe commands.

- Checking the Subset Response

Use the following command to preview the output before executing the subset operation:
```bash
copernicusmarine subset -i cmems_mod_arc_phy_my_hflux_P1M-m -y 25000 -Y 850000 -x -10000 -X 1800000 --dry-run --dataset-part originalGrid 
```
The `--dry-run` flag allows for validation of the requested subset parameters without modifying or downloading data, helping to confirm expected results.

- Inspecting Dataset Metadata

To obtain detailed metadata about the units, you can also use the describe command. Here the outputs will be stored in a file called "describe_output.json". 

```bash
copernicusmarine describe -i cmems_mod_arc_phy_my_hflux_P1M-m -e units > describe_output.json
```

# Examples

## Downloading and Plotting Data in Geographic Coordinate System in Python

We can use the following command to retrieve chlorophyll concentration, zooplankton carbon density and dissolved oxygen concentration in the artic circle in geographic coordinates: 

In [7]:
import copernicusmarine as cmt
dataset_lonlat = cmt.open_dataset(
    dataset_id="cmems_mod_arc_bgc_my_ecosmo_P1D-m", 
    variables=["chl", "zooc", "o2"], 
    minimum_latitude=64.99684, 
    maximum_latitude=90, 
    minimum_longitude=-180, 
    maximum_longitude=180,
)

ModuleNotFoundError: No module named 'copernicusmarine'

And use xarray to print the data in a tabular format:

In [ ]:
dataset_lonlat

We can see that the coordinates do not include any reference to x or y. This is the default geographical format for data. We can go a bit further and plot the data on a map. 

In [ ]:
import matplotlib.pyplot as plt 
import cartopy.crs as ccrs 
import cartopy.feature as cfeature 
 
# Select chlorophyll data for the given date 
chl = dataset_lonlat.sel(time="2021-01-01").chl.squeeze() # No need for slice() if selecting one time 
 
# Create figure and axis with PlateCarree projection 
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.NorthPolarStereo()}) 
 
# Add geographical features 
ax.set_extent([-180, 180, 60, 90], ccrs.PlateCarree()) 
ax.coastlines() 
ax.add_feature(cfeature.BORDERS, linestyle="--", linewidth=0.75) 
ax.add_feature(cfeature.LAND, edgecolor="black") 
 
# Create filled contour plot 
contour = ax.contourf(chl.longitude, chl.latitude, chl, levels=60, cmap='viridis', transform = ccrs.PlateCarree(), vmin=0.0130, vmax=0.03) 
 
# Add colorbar 
cbar = plt.colorbar(contour, ax=ax, orientation='vertical', fraction=0.046, pad=0.05) 
cbar.set_label("Chlorophyll Concentration") 
 
 
# Show the plot 
plt.show() 

And we can see that in the bounds forms circles arcs, as would be expected with geographical grids. 

## Downloading and Plotting Data in a Stereographic Grid in Python 

To retrieve data in a stereographic grid, two key adjustments must be made compared to the previous example:

- **Define Spatial Boundaries in Projected Coordinates**:
    Instead of specifying limits using `minimum_longitude` and `minimum_latitude` (along with their maximum counterparts), spatial boundaries must be defined directly in projected coordinates using `minimum_x`, `maximum_x`, `minimum_y`, and `maximum_y`.

- **Specify the originalGrid Dataset Part**:
    By default, geographical coordinates are used. To ensure data is accessed in its original stereographic format, it is necessary to explicitly specify the `originalGrid` dataset part.

In [ ]:
import copernicusmarine as cmt
# We filter the dask errors and warnings
import dask
import warnings
# Disable dask warnings by specifying 'error' as its level of logging
dask.config.set({'logging.distributed': 'error'})
# Disable all other warnings
warnings.filterwarnings("ignore")
dataset_stereographic = cmt.open_dataset(
    dataset_id="cmems_mod_arc_bgc_my_ecosmo_P1D-m", 
    variables=["chl", "zooc", "o2"], 
    minimum_y=-20, 
    maximum_y=20, 
    minimum_x=-20, 
    maximum_x=20,
    dataset_part="originalGrid"
)

And we use xarray to present the data in a tabular format:

In [ ]:
dataset_stereographic

Or even plot it on a map

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Select chlorophyll data for the specific date
chl = dataset_lonlat.sel(time="2021-01-01").chl.squeeze() # Single time-point selection

fig = plt.figure(figsize=(10,6), dpi=100)
fig.patch.set_visible(True)
fig.set_facecolor("white")

# Create an "ax" from which we add the projection
ax = plt.axes(projection=ccrs.NorthPolarStereo())


gl2 = ax.gridlines(draw_labels=True, color = 'lightgray', linewidth=0.1)
gl2.right_labels = False
gl2.top_labels = False
ax.axis('off')

ax.set_extent([-180, 180, 60, 90], crs=ccrs.PlateCarree())

# Generate chlorophyll concentration plot
contour = ax.pcolormesh(chl.x*100000, chl.y*100000, chl, cmap='viridis', transform=ccrs.NorthPolarStereo(-45), vmin=0.0130, vmax=0.03)

# Add map features
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor="lightgray")
ax.add_feature(cfeature.RIVERS, linewidth=0.5)
ax.add_feature(cfeature.LAND, linewidth=0.5, facecolor='#f1f1f1', edgecolor='lightgray', zorder=1)

# Add and format colorbar
cbar = plt.colorbar(contour, ax=ax, orientation='vertical', fraction=0.046, pad=0.05)
cbar.set_label("Chlorophyll Concentration")

plt.xlabel("x (m)")
plt.ylabel("y (m)")
# Show the plot
plt.show()

Here the boundaries appear as straight lines on a map, as would be expected from a stereographic coordinate system. 

## Downloading Data in Different Grids in the CLI

We can also retrieve the data using the Command-Line Interface.  Downloading data using the geographical coordinates is still the default and for example, we can download chloropyll, zooplankton carbon and dissolved oxygen concentration in the polar circle excluding a small disk around the pole using the following command: 

```bash
copernicusmarine subset -i cmems_mod_arc_bgc_my_ecosmo_P1D-m -o cmems_mod_arc_bgc_my_ecosmo_P1D-m_subset -v chl -v zooc -v o2 --minimum-latitude 60 --maximum-latitude 85 --minimum-longitude -180 --maximum-longitude 180 --dry-run
```

We cannot easily retrieve the same zone in stereographic coordinates. But if we wish to retrieve the same variables in a zone of a few hundred square kilometers around the the North pole, you can change the following command: 

```bash
copernicusmarine subset -i cmems_mod_arc_bgc_my_ecosmo_P1D-m -v chl -v zooc -v o2 --minimum-y -43 --maximum-y 28 --minimum-x -36 --maximum-x 38 --dataset-part originalGrid  --dry-run
```

The main difference are : 

- The indication for the bounds: 

     maximum and minimum longitude and latitude for geographical coordinates and maximum and minimum y and x for original coordinates. 

- The indication of the part with `--dataset-part originalGrid` 

If you try using original coordinates with geographical indications for the bounds, an error will be thrown and the data not downloaded: 

```bash
copernicusmarine subset -i cmems_mod_arc_bgc_my_ecosmo_P1D-m -o cmems_mod_arc_bgc_my_ecosmo_P1D-m_subset -v chl -v zooc -v o2 --minimum-latitude 60 --maximum-latitude 85 --minimum-longitude -180 --maximum-longitude 180 --dataset-part originalGrid --dry-run 
```

An error will also be thrown if you try using original coordinate indications with the default dataset part: 

```bash
copernicusmarine subset -i cmems_mod_arc_bgc_my_ecosmo_P1D-m -v chl -v zooc -v o2 --minimum-y -43 --maximum-y 28 --minimum-x -36 --maximum-x 38 --dry-run  
```

The exception appears for the shortcuts –x –X and –y –Y which are common to the two sets of coordinates. This first command will retrieve zooplankton carbon, dissolved oxygen and chlorophyll concentration for geographical coordinates:

```bash
copernicusmarine subset -i cmems_mod_arc_bgc_my_ecosmo_P1D-m -o cmems_mod_arc_bgc_my_ecosmo_P1D-m_subset -v chl -v zooc -v o2 -y 60 -Y 85 -x -180 -X 180 --dry-run
```

And this one for stereographic coordinates: 

```bash
copernicusmarine subset -i cmems_mod_arc_bgc_my_ecosmo_P1D-m -v chl -v zooc -v o2 -y -43 -Y 28 -x -36 -X 38 --dry-run --dataset-part originalGrid 
```

But they will cover very different zones